In [1]:
import imaplib
import email
from email.header import decode_header
import html2text
import re
import os
import pandas as pd
import webbrowser
import openai
import json

In [2]:
# Creating a folder
def clean(text):
    return "".join(c if c.isalnum() else "_" for c in text)

In [3]:
email_user = 'youremail@outlook.com'
email_pass = 'yourpassword'
folder_name = 'Pedidos'

# Connect to the IMAP server
mail = imaplib.IMAP4_SSL("outlook.office365.com")

# Log in to your Outlook account
mail.login(email_user, email_pass)

# Select the specific folder
mail.select(folder_name)

# Search for all emails in the selected folder
status, messages = mail.search(None, "ALL")
messages = messages[0].split()

In [4]:
emails_list = []
html_converter = html2text.HTML2Text()

# Define the clean_text function
def clean_text(text):
    # Remove extra whitespaces
    cleaned_text = re.sub(r'  ', '', text)
    # Remove special characters and formatting
    cleaned_text = re.sub(r'\*|\-|\\|__', '', cleaned_text)
    return cleaned_text.strip()

def extract_email_address(from_address):
    email_regex = r"[\w\.-]+@[\w\.-]+\.\w+"
    match = re.search(email_regex, from_address)
    return match.group(0) if match else None

# function to extract the date from the email
# and return in it in the final format
date_regex = r"\d{1,2}\s\w{3}\s\d{4}"

def get_date_from_email(message):
    date_extrated, encoding = decode_header(msg["date"])[0]
    # apllying the regex filter
    final_date = re.search(date_regex, date_extrated)
    return final_date.group()

for message_id in messages:
    # fetch the email message by ID
    res, msg = mail.fetch(message_id, "(RFC822)")
    for response_part in msg:
        if isinstance(response_part, tuple):
            # parse a bytes email into a message object
            msg = email.message_from_bytes(response_part[1])
            # extracting date from the email information
            date = get_date_from_email(msg)
            # decode the email subject
            subject, encoding = decode_header(msg["subject"])[0]
            if isinstance(subject, bytes):
                # if it's bytes, decode to str
                subject = subject.decode(encoding)
            # decode email sender
            From, encoding = decode_header(msg.get("from"))[0]
            if isinstance(From, bytes):
                From = From.decode(encoding)
            # Extract the email address from the "From" field
            from_address = msg["From"]
            email_address = extract_email_address(from_address)
            # Initialize an empty dictionary to store email information
            email_info = {"Date": date, "Subject": subject, "From": email_address, "Body": ""}
            # if the email message is multipart
            if msg.is_multipart():
                # iterate over email parts
                for part in msg.walk():
                    # extract content type of email
                    content_type = part.get_content_type()
                    content_disposition = str(part.get("Content-Disposition"))
                    try:
                        # get the email body
                        body = html_converter.handle(part.get_payload(decode=True).decode())
                        # cleaned_body = clean_text(body)  # Removido
                        email_info["Body"] = body.lower()  # Modificado
                    except:
                        pass
                    if content_type == "text/plain" and "attachment" not in content_disposition:
                        # print text/plain emails and skip attachments
                        print(body)
                    elif "attachment" in content_disposition:
                        # download attachment
                        filename = part.get_filename()
                        if filename:
                            folder_name = clean(subject)
                            if not os.path.isdir(folder_name):
                                # make a folder for this email (named after the subject)
                                os.mkdir(folder_name)
                            filepath = os.path.join(folder_name, filename)
                            # download attachment and save it
                            open(filepath, "wb").write(part.get_payload(decode=True))
            else:
                # extract content type of email
                content_type = msg.get_content_type()
                # get the email body
                body = html_converter.handle(msg.get_payload(decode=True).decode())
                # cleaned_body = clean_text(body)  # Removido
                email_info["Body"] = body.lower()  # Modificado
                if content_type == "text/plain":
                    # print only text email parts
                    print(body)
            if content_type == "text/html":
                # if it's HTML, create a new HTML file and open it in the browser
                folder_name = clean(subject)
                if not os.path.isdir(folder_name):
                    # make a folder for this email (named after the subject)
                    os.mkdir(folder_name)
                filename = "index.html"
                filepath = os.path.join(folder_name, filename)
                # write the file
                open(filepath, "w").write(body)
            print("="*100)
            emails_list.append(email_info)

# Create a DataFrame from the list of dictionaries
emails_table = pd.DataFrame(emails_list)

Boa tarde, Por favor enviem a V/ melhor cotação para esta carga: The material
will be ready starting from tomorrow January,09. 1 pallet cm 80x120x60 h Kg
120 Our warehouse time are : 08:30-12:30/13:30/17:30. Desde: 31030 Carbonera -
TV ITÁLIA Para entrega em: 3750-792 TROFA AGD Obrigado! Melhores Cumprimentos
/ Kind regards, Eduardo


Bom dia, Solicito cotação para envio de palete para Holanda com recolha no
nosso armazém na Maia. 1 Palete Comprimento 120 cm Largura 80 cm Altura 30 cm
Peso 175 kg Material sobreponível Valor da mercadoria para seguro: 300€ Morada
de carga: Rua de catassol 1644 4470-842 Maia Morada de descarga: Achterste
Weide 6 6821 SH Duiven Netherlands Obrigada. Com os melhores cumprimentos, Ana


Bom dia, Venho por este meio solicitar uma nova cotação: \- Paletes: 1 \-
Medidas – 60X50X40 \- Peso - 108kgs LOCAL DE CARGA: 24043 ITÁLIA Atentamente
Marta Santana


Boa tarde Vimos por este meio solicitar cotação para o seguinte transporte: 1
Europalet - 1.80m altura - 400

In [5]:
emails_table

,Date,Subject,From,Body
0,19 Feb 2024,Cotação,inesbarbara189@gmail.com,"\n\nboa tarde,\n\npor favor enviem a v/ melhor..."
1,19 Feb 2024,Cotação Holanda,inesbarbara189@gmail.com,"\n\nbom dia,\n\n \n\nsolicito cotação para en..."
2,19 Feb 2024,Cotação,inesbarbara189@gmail.com,"\n\nbom dia,\n\n\n\nvenho por este meio solici..."
3,19 Feb 2024,"Pedido de cotação - Veracruz Industries, Lda",inesbarbara189@gmail.com,\n\nboa tarde\n\n \n\nvimos por este meio sol...
4,19 Feb 2024,Cotação Aústria,daniela.nogueira.work@gmail.com,\n\nboa tarde sr barbosa\n\n\n\npara recolha a...
5,19 Feb 2024,Pedido de orçamento - saída a 30/11/2024,inesbarbara189@gmail.com,"\n\nboa tarde,\n\n \n\nsolicito os seguintes ..."
6,19 Feb 2024,Cotação,daniela.nogueira.work@gmail.com,"\n\nbom dia,\n\nespero que se encontre bem.\n\..."


In [6]:
i=0
emails_table['Body_new']="merge"
for a in emails_table['Subject']:
    emails_table['Body_new'][i]= a + emails_table['Body'][i]
    i=i+1
emails_table

,Date,Subject,From,Body,Body_new
0,19 Feb 2024,Cotação,inesbarbara189@gmail.com,"\n\nboa tarde,\n\npor favor enviem a v/ melhor...","Cotação\n\nboa tarde,\n\npor favor enviem a v/..."
1,19 Feb 2024,Cotação Holanda,inesbarbara189@gmail.com,"\n\nbom dia,\n\n \n\nsolicito cotação para en...","Cotação Holanda\n\nbom dia,\n\n \n\nsolicito ..."
2,19 Feb 2024,Cotação,inesbarbara189@gmail.com,"\n\nbom dia,\n\n\n\nvenho por este meio solici...","Cotação\n\nbom dia,\n\n\n\nvenho por este meio..."
3,19 Feb 2024,"Pedido de cotação - Veracruz Industries, Lda",inesbarbara189@gmail.com,\n\nboa tarde\n\n \n\nvimos por este meio sol...,"Pedido de cotação - Veracruz Industries, Lda\n..."
4,19 Feb 2024,Cotação Aústria,daniela.nogueira.work@gmail.com,\n\nboa tarde sr barbosa\n\n\n\npara recolha a...,Cotação Aústria\n\nboa tarde sr barbosa\n\n\n\...
5,19 Feb 2024,Pedido de orçamento - saída a 30/11/2024,inesbarbara189@gmail.com,"\n\nboa tarde,\n\n \n\nsolicito os seguintes ...",Pedido de orçamento - saída a 30/11/2024\n\nbo...
6,19 Feb 2024,Cotação,daniela.nogueira.work@gmail.com,"\n\nbom dia,\n\nespero que se encontre bem.\n\...","Cotação\n\nbom dia,\n\nespero que se encontre ..."


In [7]:
emails_table.drop(['Body','Subject'], axis=1,inplace=True)
emails_table.rename(columns={'Body_new':'Body'},inplace=True)
# Set Email_id as the original index
emails_table.reset_index(inplace=True)  # Reset index and make it a column
emails_table.rename(columns={'index': 'Email_ID'}, inplace=True)  # Rename the column
emails_table.set_index('Email_ID', inplace=True)
emails_table

,Date,From,Body
Email_ID,,,
0,19 Feb 2024,inesbarbara189@gmail.com,"Cotação\n\nboa tarde,\n\npor favor enviem a v/..."
1,19 Feb 2024,inesbarbara189@gmail.com,"Cotação Holanda\n\nbom dia,\n\n \n\nsolicito ..."
2,19 Feb 2024,inesbarbara189@gmail.com,"Cotação\n\nbom dia,\n\n\n\nvenho por este meio..."
3,19 Feb 2024,inesbarbara189@gmail.com,"Pedido de cotação - Veracruz Industries, Lda\n..."
4,19 Feb 2024,daniela.nogueira.work@gmail.com,Cotação Aústria\n\nboa tarde sr barbosa\n\n\n\...
5,19 Feb 2024,inesbarbara189@gmail.com,Pedido de orçamento - saída a 30/11/2024\n\nbo...
6,19 Feb 2024,daniela.nogueira.work@gmail.com,"Cotação\n\nbom dia,\n\nespero que se encontre ..."


In [8]:
import json
import pandas as pd

openai.api_key = 'yourAPIkey"
openai.api_base = 'https://yourlink/'
openai.api_version = "date"
openai.api_type = "azure"
 
deployment_name='deploymentname'
 
def get_completion(prompt):
    response = openai.ChatCompletion.create(
        engine=deployment_name,
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return response['choices'][0]['message']['content']

# Inicialize o dicionário para armazenar os resultados finais
results_dict = {}

# Loop sobre os emails
for index, row in emails_table.iterrows():
    text2 = row['Body']
    prompt = f"""    
    O objetivo é realizar e retornar as seguintes tarefas: 
    Verifique se o email contém as seguintes informações essenciais para a elaboração de um orçamento: 
    1 -  Loaddress: a morada a partir da qual as mercadorias estão a ser enviadas; 
    2 -  Consignor: para quem as mercadorias estão a ser enviadas: pessoa/empresa; 
    3 -  Unloadress: a morada de destino das mercadorias;
    4 -  Loading Date: a data em que as mercadorias estão agendadas para serem carregadas para transporte,resposta em formato dia/mês - dd/mm;
    5 -  Unloading Date: a data em que as mercadorias estão agendadas para serem descarregadas no destino,resposta em formato dia/mês - dd/mm;
    6 -  Package Type: o tipo de embalagem utilizado, por exemplo, paletes (em casos de aparecer "pallet" assume como "paletes" na resposta), contentores, bidões ou caixas);
    7 -  Number of Packages: o número de embalagens a serem transportadas;
    8 -  Commodities: descrição das mercadorias a serem transportadas (Produtos Farmacêuticos, Eletrónicos, Alimentos, Produtos Químicos ou Materiais de Construção);
    9 -  Dimensions(Height x Width x Length): apresenta a resposta, por exemplo, neste formato "30x80x120";
    10 - Volume(m³): se o volume das mercadorias não for fornecido no email, mas as dimensões estiverem disponíveis, calcule o volume multiplicando,sem arredondamentos, altura x largura x comprimento e apresente o resultado exato em metros cúbicos;
    11 - Gross Weight(kg): resposta sempre o formato, por exemplo, "120"; 
    12 - Plan Characteristics: características especiais do plano de transporte, como envolvimento de mercadorias perigosas, substâncias inflamáveis ou sujeitas a regulamentações especiais;
    13 - Additional services: quaisquer serviços extras necessários, como controlo de temperatura, rastreamento em tempo real, armazenagem temporária ou medidas de segurança;
    14 - Ensurance: deseja/tem Seguro de carga, resposta "sim" ou "não" e em caso de existir no email o Valor da mercadoria, assumir logo "sim";
    15 - Goods Value: o valor das mercadorias a serem transportadas;
    16 - Delivery instructions: instruções de entrega. 

    Apresente os resultados em formato JSON, incluindo as seguintes chaves:
    1 -  Loaddress 
    2 -  Consignor
    3 -  Unloadress
    4 -  Loading Date
    5 -  Unloading Date
    6 -  Package Type 
    7 -  Number of Packages
    8 -  Commodities
    9 -  Dimensions(Height x Width x Length)
    10 - Volume(m³)
    11 - Gross Weight(kg)
    12 - Plan Characteristics
    13 - Additional services
    14 - Ensurance 
    15 - Goods Value
    16 - Delivery instructions
    
    Se alguma informação estiver ausente, atribua sempre o valor "sem informação" à chave correspondente.
    Separe os pedidos em diferentes objetos, Pedido 1, Pedido 2 ... e assim sucessivamente.
    
    Quero que tenhas atenção a alguns detalhes adicionais:
    Loaddress: quando aparecer as palavras "desde" ou "local de carga" ou "saída a" ou "com recolha em" assume estarem relacionadas com a morada a partir da qual as mercadorias estão a ser enviadas;
    Unloadress: quando aparecer as palavras "para" ou "local de descarga" assume estarem relacionadas com a morada de destino das mercadorias;
    Loading Date: quando aparecer as palavras "will be ready on/from" assume estarem relacionadas a data em que as mercadorias estão agendadas para serem carregadas para transporte;
    Text: <{text2}>
    """
    # Obter a resposta
    response = get_completion(prompt)
    
    # Converter a resposta JSON em um dicionário
    response_dict = json.loads(response)
    
    # Adicionar o dicionário à lista de pedidos de entrega para este email
    if index not in results_dict:
        results_dict[index] = []
    results_dict[index].append(response_dict)

# Converter o dicionário para DataFrame
llm_df = pd.DataFrame(results_dict.items(), columns=['Email_ID', 'Pedidos'])

# Exibir o DataFrame final
llm_df

,Email_ID,Pedidos
0,0,[{'Pedido 1': {'Loaddress': '31030 carbonera -...
1,1,[{'Pedido 1': {'Loaddress': 'rua de catassol 1...
2,2,"[{'Pedido 1': {'Loaddress': '24043 itália', 'C..."
3,3,[{'Pedido 1': {'Loaddress': 'gwiker en 115 - k...
4,4,"[{'Pedido 1': {'Loaddress': 'hotel madlein, ma..."
5,5,"[{'Pedido 1': {'Loaddress': 'nosso armazém', '..."
6,6,"[{'Pedido 1': {'Loaddress': 'sem informação', ..."


In [9]:
llm_df['Pedidos'][0]

[{'Pedido 1': {'Loaddress': '31030 carbonera - tv itália',
   'Consignor': 'sem informação',
   'Unloadress': '3750-792 trofa agd',
   'Loading Date': '09/01',
   'Unloading Date': 'sem informação',
   'Package Type': 'paletes',
   'Number of Packages': '1',
   'Commodities': 'sem informação',
   'Dimensions(Height x Width x Length)': '60x80x120',
   'Volume(m³)': '0.576',
   'Gross Weight(kg)': '120',
   'Plan Characteristics': 'sem informação',
   'Additional services': 'sem informação',
   'Ensurance': 'sem informação',
   'Goods Value': 'sem informação',
   'Delivery instructions': 'sem informação'}}]

In [10]:
emails_table['Body'][0]

'Cotação\n\nboa tarde,\n\npor favor enviem a v/ melhor cotação para esta carga:\n\n  \n\nthe material will be ready starting from tomorrow january,09.\n\n1 pallet cm 80x120x60 h\n\nkg 120\n\nour warehouse time are :\n\n08:30-12:30/13:30/17:30.\n\n  \n\ndesde:\n\n31030 carbonera - tv itália\n\n  \n\npara entrega em:\n\n3750-792 trofa agd\n\nobrigado!\n\nmelhores cumprimentos / kind regards,\n\neduardo\n\n'

In [11]:
import pandas as pd
# Iterando sobre cada linha na coluna 'Pedido' e expandindo os dicionários
rows = []
dfs = []

# Iterar sobre cada linha do DataFrame
for index, row in llm_df.iterrows():
    # Iterar sobre cada pedido na lista de pedidos
    for a in llm_df['Pedidos'][index]:
        # Criar um DataFrame a partir do pedido
        b = pd.DataFrame(a).T
        # Adicionar uma nova coluna com o ID do e-mail
        email_id = llm_df['Email_ID'][index]
        b['Email_ID'] = email_id
        # Definir a coluna do ID do e-mail como o índice
        b.set_index(b['Email_ID'], inplace=True)
        b.drop('Email_ID', axis=1, inplace=True)
        # Adicionar o DataFrame resultante à lista
        dfs.append(b)

# Criando um novo DataFrame com as linhas expandidas
formatted_df = pd.DataFrame(rows)
result_2 = pd.concat(dfs)

# DataFrame final
df_final=pd.concat([formatted_df,result_2])
#reorganize columns
df_final=df_final[['Loaddress',
   'Consignor',
   'Unloadress',
   'Loading Date',
   'Unloading Date',
   'Package Type',
   'Number of Packages',
   'Commodities',
   'Dimensions(Height x Width x Length)',
   'Volume(m³)',
   'Gross Weight(kg)',
   'Plan Characteristics',
   'Additional services',
   'Ensurance',
   'Goods Value',
   'Delivery instructions']]
# Exibir o DataFrame final
df_final

,Loaddress,Consignor,Unloadress,Loading Date,Unloading Date,Package Type,Number of Packages,Commodities,Dimensions(Height x Width x Length),Volume(m³),Gross Weight(kg),Plan Characteristics,Additional services,Ensurance,Goods Value,Delivery instructions
Email_ID,,,,,,,,,,,,,,,,
0,31030 carbonera - tv itália,sem informação,3750-792 trofa agd,09/01,sem informação,paletes,1,sem informação,60x80x120,0.576,120,sem informação,sem informação,sem informação,sem informação,sem informação
1,"rua de catassol 1644, 4470-842 maia",sem informação,"achterste weide 6, 6821 sh duiven, netherlands",sem informação,sem informação,paletes,1,sem informação,30x80x120,0.288,175,sem informação,sem informação,sim,300€,sem informação
2,24043 itália,sem informação,sem informação,sem informação,sem informação,paletes,1,sem informação,60x50x40,0.12,108,sem informação,sem informação,sem informação,sem informação,sem informação
3,"gwiker en 115 - km 4,85 - fração d, 2500-064 c...",sem informação,"landgasthof leuen, daniel buerkli, birmensdorf...",sem informação,sem informação,paletes,1,sem informação,1.80xsem informaçãoxsem informação,sem informação,400,sem informação,sem informação,sem informação,sem informação,sem informação
4,"hotel madlein, madleinweg 2, 6561 ischgl - áus...",sem informação,sem informação,recolha amanhã,sem informação,caixas,10,sem informação,60x40x50,0.12,130,sem informação,com recolha em,sem informação,sem informação,sem informação
5,nosso armazém,sem informação,"jean yves léonard, 166, rue du faubourg saint ...",30/11,sem informação,paletes,3,sem informação,110x120x140,1.848,250,carga não sobreponível,"controlo de temperatura, rastreamento em tempo...",sim,9138.00€,delivery via trail lifttruck. please inform cl...
5,nosso armazém,sem informação,"schreinerei kraft e.k., obere grüben 10, d-978...",sem informação,sem informação,palete,1,sem informação,100x120x110,1.32,150,sem informação,sem informação,sim,2276.00€,delivery via international truck. please infor...
5,nosso armazém,sem informação,"götz arndt gmbh, tückingstr. 40, 41460 neuss, ...",sem informação,sem informação,"caixa de madeira, paletes",2,sem informação,"160x80x80, 160x80x70",1.792,400,sem informação,sem informação,sim,4498.00€,delivery via trail lift truck. please inform c...
6,sem informação,meubles.barriol@orange.fr,sem informação,sem informação,sem informação,caixas,1,sem informação,20.5x22x7,0.00377,1.5,sem informação,sem informação,sem informação,sem informação,sem informação


In [12]:
# Merge the DataFrames on index
merged_df = pd.merge(emails_table.drop('Body', axis=1), df_final, left_index=True, right_index=True)

# Display the merged DataFrame
merged_df

,Date,From,Loaddress,Consignor,Unloadress,Loading Date,Unloading Date,Package Type,Number of Packages,Commodities,Dimensions(Height x Width x Length),Volume(m³),Gross Weight(kg),Plan Characteristics,Additional services,Ensurance,Goods Value,Delivery instructions
Email_ID,,,,,,,,,,,,,,,,,,
0,19 Feb 2024,inesbarbara189@gmail.com,31030 carbonera - tv itália,sem informação,3750-792 trofa agd,09/01,sem informação,paletes,1,sem informação,60x80x120,0.576,120,sem informação,sem informação,sem informação,sem informação,sem informação
1,19 Feb 2024,inesbarbara189@gmail.com,"rua de catassol 1644, 4470-842 maia",sem informação,"achterste weide 6, 6821 sh duiven, netherlands",sem informação,sem informação,paletes,1,sem informação,30x80x120,0.288,175,sem informação,sem informação,sim,300€,sem informação
2,19 Feb 2024,inesbarbara189@gmail.com,24043 itália,sem informação,sem informação,sem informação,sem informação,paletes,1,sem informação,60x50x40,0.12,108,sem informação,sem informação,sem informação,sem informação,sem informação
3,19 Feb 2024,inesbarbara189@gmail.com,"gwiker en 115 - km 4,85 - fração d, 2500-064 c...",sem informação,"landgasthof leuen, daniel buerkli, birmensdorf...",sem informação,sem informação,paletes,1,sem informação,1.80xsem informaçãoxsem informação,sem informação,400,sem informação,sem informação,sem informação,sem informação,sem informação
4,19 Feb 2024,daniela.nogueira.work@gmail.com,"hotel madlein, madleinweg 2, 6561 ischgl - áus...",sem informação,sem informação,recolha amanhã,sem informação,caixas,10,sem informação,60x40x50,0.12,130,sem informação,com recolha em,sem informação,sem informação,sem informação
5,19 Feb 2024,inesbarbara189@gmail.com,nosso armazém,sem informação,"jean yves léonard, 166, rue du faubourg saint ...",30/11,sem informação,paletes,3,sem informação,110x120x140,1.848,250,carga não sobreponível,"controlo de temperatura, rastreamento em tempo...",sim,9138.00€,delivery via trail lifttruck. please inform cl...
5,19 Feb 2024,inesbarbara189@gmail.com,nosso armazém,sem informação,"schreinerei kraft e.k., obere grüben 10, d-978...",sem informação,sem informação,palete,1,sem informação,100x120x110,1.32,150,sem informação,sem informação,sim,2276.00€,delivery via international truck. please infor...
5,19 Feb 2024,inesbarbara189@gmail.com,nosso armazém,sem informação,"götz arndt gmbh, tückingstr. 40, 41460 neuss, ...",sem informação,sem informação,"caixa de madeira, paletes",2,sem informação,"160x80x80, 160x80x70",1.792,400,sem informação,sem informação,sim,4498.00€,delivery via trail lift truck. please inform c...
6,19 Feb 2024,daniela.nogueira.work@gmail.com,sem informação,meubles.barriol@orange.fr,sem informação,sem informação,sem informação,caixas,1,sem informação,20.5x22x7,0.00377,1.5,sem informação,sem informação,sem informação,sem informação,sem informação
